## Diagnóstico com Deep Autoencoder

**Criado por: Leonardo Franco de Godói**


**Data: 27/09/2019**

--------------------------------------------------------------------------------------------------------------------------

#### Descrição:
O dataset utilizado, contendo dados obtidos de rolamentos, está disponível em Case Western Reserve University bearing data center, através do link: 

http://csegroups.case.edu/bearingdatacenter/pages/welcome-case-western-reserve-university-bearing-data-center-website.

O dataset contempla três tipos de falhas mecânicas, associadas a:

- Elementos rolantes.
- Anel interno.
- Anel externo.

Além disso, o dataset é também dividido entre duas condições de operação em termos de velocidade de rotação - configuração de carga:

- 20-0
- 30-2

Os dados foram obtidos através de 8 sensores:

- 1 (vibração do motor).
- 2, 3, 4 (vibração da caixa de engrenagens planetária nas direções X, Y e Z).
- 5 (torque do motor).
- 6, 7, 8 (vibração da caixa de engrenagens paralela nas direções X, Y e Z).

Esta implementação visa o diagnóstico de um rolamento através da classificação de sua condição entre 5 possíveis estados:

- Saudável (Health).
- Falha no anel interno (Inner).
- Falha no anel externo (Outer).
- Falha de elemento rolante (Ball).
- Falha combinada dos anéis interno e externo (Comb).

Um Deep Autoencoder (Deep AE) com a configuração (256-128-64-32-64-128-256) é aplicado sobre um dataset misto, contendo ocorrências de todos os estados mencionados. Após o treinamento do Autoencoder, uma camada para classificação é conectada ao gargalo (contendo a informação comprimida) e um novo treino, desta vez supervisionado, é realizado.

---------------------------------------------------------------------------------------------------------------------------

*Importando os pacotes necessários.*

In [2]:
import pandas as pd
import numpy as np
from keras.layers import Input, Dense
from keras.models import Model
import keras
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


*Informando o diretório dos datasets originais.*

In [3]:
# Definindo o diretório dos datasets originais
dir_health = 'health_30_2.csv'
dir_inner = 'inner_30_2.csv'
dir_outer = 'outer_30_2.csv'
dir_ball = 'ball_30_2.csv'
dir_comb = 'comb_30_2.csv'

*Número de classes (possíveis estados).*

In [4]:
num_classes = 5

*Definindo os parâmetros da rede neural.*

In [5]:
hidden_layer1 = 256
hidden_layer2 = 128
hidden_layer3 = 64
hidden_layer4 = 32
NUM_EPOCHS = 100
BATCH_SIZE = 1024
act_func = 'relu'

*Carregando os datasets originais (8 sensores x 1048560 medições).*

- *Dataset saudável (health).*

- *Dataset com falha no anel interno (inner).*

- *Dataset com falha no anel externo (outer).*

- *Dataset com falha nos elementos rolantes (ball).*

- *Dataset com falha combinada dos anéis (comb).*

In [6]:
dataset_health = pd.read_csv(dir_health, sep='\t', header=-1)
dataset_inner = pd.read_csv(dir_inner, sep='\t', header=-1)
dataset_outer = pd.read_csv(dir_outer, sep='\t', header=-1)
dataset_ball = pd.read_csv(dir_ball, sep='\t', header=-1)
dataset_comb = pd.read_csv(dir_comb, sep='\t', header=-1)

*Removendo a última coluna (vazia).*

In [7]:
dataset_health = dataset_health.iloc[:,:-1] 
dataset_inner = dataset_inner.iloc[:,:-1]
dataset_outer = dataset_outer.iloc[:,:-1]
dataset_ball = dataset_ball.iloc[:,:-1]
dataset_comb = dataset_comb.iloc[:,:-1]

*Definindo rótulos para as colunas e ajustando as linhas.*

In [9]:
dataset_health.columns = ['Sensor 1', 'Sensor 2', 'Sensor 3', 'Sensor 4',
                          'Sensor 5', 'Sensor 6', 'Sensor 7', 'Sensor 8']
dataset_inner.columns = dataset_health.columns
dataset_inner.index = dataset_health.index
dataset_outer.columns = dataset_health.columns
dataset_outer.index = dataset_health.index
dataset_ball.columns = dataset_health.columns
dataset_ball.index = dataset_health.index
dataset_comb.columns = dataset_health.columns
dataset_comb.index = dataset_health.index

*Definindo os conjuntos de treinamento e validação.*

In [10]:
train_size = 50000
test_size = 20000
X_train = dataset_health[:train_size].append([dataset_inner[:train_size],
                                          dataset_outer[:train_size],
                                          dataset_ball[:train_size],
                                          dataset_comb[:train_size]])
X_test = dataset_health[:test_size].append([dataset_inner[:test_size],
                                          dataset_outer[:test_size],
                                          dataset_ball[:test_size],
                                          dataset_comb[:test_size]])
y_train = np.repeat(np.array([0,1,2,3,4]), train_size)
y_test = np.repeat(np.array([0,1,2,3,4]), test_size)

*Convertendo os vetores de classes em matrizes binárias.*

In [11]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

*Estruturando o modelo do Autoencoder.*

In [12]:
my_input = Input(shape=(X_train.shape[1],))
encoded = Dense(hidden_layer1, activation=act_func)(my_input)
encoded = Dense(hidden_layer2, activation=act_func)(encoded)
encoded = Dense(hidden_layer3, activation=act_func)(encoded)
encoded = Dense(hidden_layer4, activation=act_func)(encoded) # Gargalo
decoded = Dense(hidden_layer3, activation=act_func)(encoded)
decoded = Dense(hidden_layer2, activation=act_func)(decoded)
decoded = Dense(hidden_layer1, activation=act_func)(decoded)
decoded = Dense(X_train.shape[1], activation='sigmoid')(decoded)
autoencoder = Model(my_input, decoded)

Instructions for updating:
Colocations handled automatically by placer.


*Compilando o modelo.*

In [13]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

*Criando um mecanismo de Early Stopping baseado na perda para evitar overfitting.*

In [14]:
es_loss = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

*Treinando o Autoencoder.*

In [15]:
autoencoder.fit(X_train, X_train,
                epochs=NUM_EPOCHS,
                batch_size=BATCH_SIZE,
                callbacks=[es_loss],
                shuffle=True,
                validation_data=(X_test, X_test))

Instructions for updating:
Use tf.cast instead.
Train on 250000 samples, validate on 100000 samples
Epoch 1/100
250000/250000 [==============================] - 5s 20us/step - loss: -0.9130 - val_loss: -1.7650
Epoch 2/100
250000/250000 [==============================] - 4s 15us/step - loss: -1.0647 - val_loss: -1.7650
Epoch 00002: early stopping


*Inserindo a camada de classificação.*

In [16]:
output = Dense(num_classes, activation='softmax')(encoded)

*Redefinindo o modelo da rede neural.*

In [17]:
autoencoder = Model(my_input, output)

*Compilando o novo modelo.*

In [18]:
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

*Treinando de forma supervisionada.*

In [19]:
autoencoder.fit(X_train,
                y_train,
                epochs=NUM_EPOCHS,
                batch_size=BATCH_SIZE,
                validation_data=(X_test, y_test))

Train on 250000 samples, validate on 100000 samples
Epoch 1/100
250000/250000 [==============================] - 6s 22us/step - loss: 1.1606 - acc: 0.4519 - val_loss: 0.7680 - val_acc: 0.6090
Epoch 2/100
250000/250000 [==============================] - 3s 13us/step - loss: 0.7676 - acc: 0.5641 - val_loss: 0.7246 - val_acc: 0.6102
Epoch 3/100
250000/250000 [==============================] - 5s 18us/step - loss: 0.7470 - acc: 0.5734 - val_loss: 0.7188 - val_acc: 0.6041
Epoch 4/100
250000/250000 [==============================] - 3s 13us/step - loss: 0.7366 - acc: 0.5792 - val_loss: 0.7000 - val_acc: 0.6167
Epoch 5/100
250000/250000 [==============================] - 3s 13us/step - loss: 0.7309 - acc: 0.5827 - val_loss: 0.6901 - val_acc: 0.6219
Epoch 6/100
250000/250000 [==============================] - 3s 14us/step - loss: 0.7280 - acc: 0.5857 - val_loss: 0.7015 - val_acc: 0.6046
Epoch 7/100
250000/250000 [==============================] - 4s 15us/step - loss: 0.7216 - acc: 0.5928 - val

Epoch 59/100
250000/250000 [==============================] - 4s 15us/step - loss: 0.6424 - acc: 0.6765 - val_loss: 0.6045 - val_acc: 0.7155
Epoch 60/100
250000/250000 [==============================] - 3s 13us/step - loss: 0.6421 - acc: 0.6761 - val_loss: 0.5964 - val_acc: 0.7190
Epoch 61/100
250000/250000 [==============================] - 3s 13us/step - loss: 0.6421 - acc: 0.6761 - val_loss: 0.5956 - val_acc: 0.7199
Epoch 62/100
250000/250000 [==============================] - 3s 14us/step - loss: 0.6412 - acc: 0.6772 - val_loss: 0.6126 - val_acc: 0.7117
Epoch 63/100
250000/250000 [==============================] - 4s 15us/step - loss: 0.6409 - acc: 0.6776 - val_loss: 0.6099 - val_acc: 0.7075
Epoch 64/100
250000/250000 [==============================] - 3s 13us/step - loss: 0.6406 - acc: 0.6773 - val_loss: 0.6031 - val_acc: 0.7118
Epoch 65/100
250000/250000 [==============================] - 3s 13us/step - loss: 0.6404 - acc: 0.6782 - val_loss: 0.6008 - val_acc: 0.7200
Epoch 66/100


*Avaliando o desempenho da rede.*

In [20]:
score = autoencoder.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.5893179732513427
Test accuracy: 0.72334
